![SolidQ](https://antoniosql.github.io/images/SolidQ_Verne.png) 

# Carga de datos y revisión

In [1]:
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt


In [7]:
conn = sqlite3.connect('Tiendas24H.sqlite')
#Consideramos que un cliente nos ha abandonado si no nos compra desde hace más de 200 días. 
consulta = "select c.CodCliente, max(v.fecha) as FechaUltimaCompra, EstadoCivil, Sexo,IngresosAnuales, N.NivelAcademico, O.Ocupacion, round((julianday('now') - julianday(max(v.fecha)))) as DiasUltimaCompra, case when round((julianday('now') - julianday(max(v.fecha))))>200 then 1 else 0 end as 'Abandono' from clientes c inner join NivelAcademico N  on c.NivelAcademico = N.Codigo  inner join Ocupacion O on c.Ocupacion = o.Codigo inner join VentasCAB v on v.codcliente = C.codcliente where c.Codcliente <>'CONTADO' group by c.CodCliente,  EstadoCivil, Sexo, IngresosAnuales, N.NivelAcademico, O.Ocupacion"

df = pd.read_sql_query(consulta,conn)

In [8]:
df

,CodCliente,FechaUltimaCompra,EstadoCivil,Sexo,IngresosAnuales,NivelAcademico,Ocupacion,DiasUltimaCompra,Abandono
0,000020,2020-02-08,M,M,90000.0,Licenciatura,Profesional especializado,227.0,1
1,000026,2020-04-24,S,M,60000.0,Licenciatura,Profesional especializado,151.0,0
2,000029,2019-12-05,M,M,60000.0,Licenciatura,Profesional especializado,292.0,1
3,000047,2020-04-19,S,F,80000.0,Licenciatura,Profesional especializado,156.0,0
4,000067,2019-10-22,S,M,70000.0,Licenciatura,Profesional especializado,336.0,1
...,...,...,...,...,...,...,...,...,...
2377,Y1795344R,2019-05-11,M,F,70000.0,Estudios de postgrado,Mando intermedio,500.0,1
2378,Y2404085R,2020-01-24,S,F,90000.0,Estudios universitarios (en curso),Profesional especializado,242.0,1
2379,Y2544794L,2020-03-21,M,M,90000.0,Estudios universitarios (en curso),Profesional especializado,185.0,0
2380,Y2671043K,2020-05-01,S,F,90000.0,Estudios universitarios (en curso),Profesional especializado,144.0,0


In [13]:
caracteristicas = ["EstadoCivil", "Sexo", "IngresosAnuales","NivelAcademico", "Ocupacion"]
etiqueta="Abandono"

In [14]:
from sklearn.preprocessing import  OneHotEncoder,LabelEncoder

le=LabelEncoder()
df['EstadoCivil'] = le.fit_transform(df['EstadoCivil'])
df['Sexo'] = le.fit_transform(df['Sexo'])

In [15]:
dum_df = pd.get_dummies(df[caracteristicas], columns=["NivelAcademico","Ocupacion"] )

In [16]:

X, y = dum_df, df[etiqueta]

# Separar datos y escalar

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1 , test_size=0.3)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Árboles de Decisión


In [18]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier().fit(X_train, y_train)
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

Accuracy of Decision Tree classifier on training set: 0.83
Accuracy of Decision Tree classifier on test set: 0.80


In [19]:
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
pred = clf.predict(X_test)
print(confusion_matrix(y_test, pred))
print(classification_report(y_test, pred))

[[ 11 106]
 [ 37 561]]
              precision    recall  f1-score   support

           0       0.23      0.09      0.13       117
           1       0.84      0.94      0.89       598

    accuracy                           0.80       715
   macro avg       0.54      0.52      0.51       715
weighted avg       0.74      0.80      0.76       715



In [20]:
from sklearn.ensemble import RandomForestClassifier
 
# Crear el modelo con 100 arboles
model = RandomForestClassifier(n_estimators=300, 
                               bootstrap = True, verbose=2,
                               max_features = 'sqrt',n_jobs=4)
# a entrenar!
model.fit(X_train, y_train)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
building tree 1 of 300
building tree 2 of 300building tree 3 of 300

building tree 4 of 300
building tree 5 of 300building tree 6 of 300

building tree 7 of 300
building tree 8 of 300
building tree 9 of 300building tree 10 of 300
building tree 11 of 300

building tree 12 of 300
building tree 13 of 300building tree 14 of 300

building tree 15 of 300building tree 16 of 300

building tree 17 of 300
building tree 18 of 300building tree 19 of 300building tree 20 of 300


building tree 21 of 300building tree 22 of 300building tree 23 of 300


building tree 24 of 300
building tree 25 of 300
building tree 26 of 300
building tree 27 of 300building tree 28 of 300

building tree 29 of 300building tree 30 of 300

building tree 31 of 300
building tree 32 of 300
building tree 33 of 300
building tree 34 of 300building tree 35 of 300

building tree 36 of 300
buil

RandomForestClassifier(max_features='sqrt', n_estimators=300, n_jobs=4,
                       verbose=2)

In [21]:
model.score(X_test,y_test)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 300 out of 300 | elapsed:    0.0s finished


0.8125874125874126

In [22]:
model.score(X_train,y_train)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 300 out of 300 | elapsed:    0.0s finished


0.8332333533293341

In [23]:
model.predict_proba(X_test)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 300 out of 300 | elapsed:    0.0s finished


array([[0.2169696 , 0.7830304 ],
       [0.33454894, 0.66545106],
       [0.28239338, 0.71760662],
       ...,
       [0.09086381, 0.90913619],
       [0.08420894, 0.91579106],
       [0.01244709, 0.98755291]])

In [24]:
model.predict(X_test)

[Parallel(n_jobs=4)]: Using backend ThreadingBackend with 4 concurrent workers.
[Parallel(n_jobs=4)]: Done  33 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 154 tasks      | elapsed:    0.0s
[Parallel(n_jobs=4)]: Done 300 out of 300 | elapsed:    0.0s finished


array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1,